In [4]:
import cv2
import numpy as np
import pandas as pd
import os
from skimage.feature import hog, local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA as pca
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# Udit bhaiya Ki JAI

In [5]:
# Load the preprocessed features and labels from CSV
train_df = pd.read_csv('train_features.csv', header=None)

# Separate features and labels
X_train = train_df.iloc[:, 1:].values  # Features
y_train = train_df.iloc[:, 0].values  # Labels (assuming label is in the first column)

# Load the preprocessed features and labels from CSV
test_df = pd.read_csv('test_features.csv', header=None)

# Separate features and labels
X_test = test_df.iloc[:, 1:].values  # Features
y_test = test_df.iloc[:, 0].values  # Labels (assuming label is in the first column)

# Split the dataset into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, f1_score

# Paths to the dataset
train_dir = "Dataset/train"
test_dir = "Dataset/test"

# Load the dataset
batch_size = 32
img_size = (48, 48)  # Based on FER2013 image dimensions

train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_dir,
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size,
    shuffle=True
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_dir,
    label_mode="int",
    image_size=img_size,
    batch_size=batch_size
)

# Normalize the images
normalization_layer = tf.keras.layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))
test_dataset = test_dataset.map(lambda x, y: (normalization_layer(x), y))

# Extract the labels for later use
train_labels = np.concatenate([y.numpy() for _, y in train_dataset])
test_labels = np.concatenate([y.numpy() for _, y in test_dataset])

# Load a pre-trained CNN model (VGG16 in this example)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(48, 48, 3))
x = tf.keras.layers.Flatten()(base_model.output)
feature_extractor = Model(inputs=base_model.input, outputs=x)

# Extract feature vectors for the train and test sets
train_features = []
for images, _ in train_dataset:
    train_features.append(feature_extractor.predict(images))
train_features = np.concatenate(train_features)

test_features = []
for images, _ in test_dataset:
    test_features.append(feature_extractor.predict(images))
test_features = np.concatenate(test_features)

# Fit GMM on the feature vectors
num_classes = len(train_dataset.class_names)
gmm = GaussianMixture(n_components=num_classes, random_state=42)
gmm.fit(train_features)

# Predict on test feature vectors
test_predictions = gmm.predict(test_features)

# Map GMM cluster predictions to class labels
label_encoder = LabelEncoder()
true_labels_encoded = label_encoder.fit_transform(train_dataset.class_names)
mapped_predictions = label_encoder.inverse_transform(test_predictions)

# Evaluate the model
accuracy = accuracy_score(test_labels, test_predictions)
f1 = f1_score(test_labels, test_predictions, average='weighted')
report = classification_report(test_labels, test_predictions)

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"F1 Score: {f1:.2f}")
print("Classification Report:")
print(report)


^C
Defaulting to user installation because normal site-packages is not writeable
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached h5py-3.12.1-cp311-cp311-win_amd64.whl.metadata (2.5 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/390.2 MB ? eta -:--:--
   ---------------------------------------- 2.1/390.2 MB 13.0 MB/s eta 0:00:30
    --------------------------------------- 6.3/390.2 MB 17.5 MB/s eta 0:00:22
   - -------------------------------------- 10.2/390.2 MB 18.2 MB/s eta 0:00:21
   - -------------------------------------- 13.9/390.2 MB 17.8 MB/s eta 0:00:22
   - -------------------------------------- 17.8/390.2 MB 18.1 MB/s eta 0:00:21
   -- ------------------------------------- 22.3/390.2 M


[notice] A new release of pip is available: 24.3 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip
